In [ ]:
import wandb
import numpy as np
import torch
import torch.nn as nn
from pathlib import Path
from model_fusion.config import BASE_DATA_DIR, CHECKPOINT_DIR
from model_fusion.datasets import DataModuleType
from model_fusion.models import ModelType
from model_fusion.models.lightning import BaseModel 
from Experiments import lmc_experiment
from Experiments import baselines_experiment
from Experiments import otfusion_experiment
import pyhessian as hessian
from Experiments import pyhessian_experiment

# set seed for numpy based calculations
NUMPY_SEED = 100
np.random.seed(NUMPY_SEED)

In [ ]:
print("------- Loading models -------")

# select wandb run names
runA = 'bbecqkxs'
runB = 'k9q16yq1'

api = wandb.Api()
run = api.run(f'model-fusion/Model Fusion/{runA}')

print(run.config)

batch_size = run.config['datamodule_hparams'].get('batch_size')

datamodule_type_str = run.config['datamodule_type'].split('.')[1].lower()
datamodule_type = DataModuleType(datamodule_type_str)
datamodule_hparams = run.config['datamodule_hparams']

model_type_str = run.config['model_type'].split('.')[1].lower()
model_type = ModelType(model_type_str)

# model_hparams = list(filter(lambda x: 'model_hparams/' in x[0], run.config.items()))
# model_hparams = {k.split('/')[1]: v for k, v in model_hparams}

model_hparams = run.config['model_hparams']

print(datamodule_hparams)
print(model_hparams)

checkpointA = f'model-fusion/Model Fusion/model-{runA}:best_k'
checkpointB = f'model-fusion/Model Fusion/model-{runB}:best_k'

run = wandb.init()

artifact = run.use_artifact(checkpointA, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelA = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")

torch.save(modelA.model.state_dict(), 'model0.ckpt')

artifact = run.use_artifact(checkpointB, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelB = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")

torch.save(modelB.model.state_dict(), 'model1.ckpt')


In [ ]:
# LMC barrier
print("------- Computing LMC barrier -------")

# # lmc_experiment.run_lmc(
#     datamodule_type=datamodule_type,
#     modelA=modelA,
#     modelB=modelB,
# )

In [ ]:
# Baselines (prediction ensembling, vanilla averaging)
print("------- Computing baselines -------")

wandb_tag = f'baselines-{runA}-{runB}'

vanilla_averaging_model = baselines_experiment.run_baselines(
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag,
)

In [ ]:
# OT model fusion + eval aligned model 
print("------- Computing model fusion -------")

wandb_tag = f"ot_model_fusion-{runA}-{runB}"

ofused_model, aligned_base_model = otfusion_experiment.run_otfusion(
    batch_size=batch_size,
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag
)

In [ ]:
# Pyhessian I (compute sharpness and eigenspectrum of base models, vanilla avg and ot fusion solutions)
print("------- Computing Pyhessian I -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=modelA)

top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues()
print("The top Hessian eigenvalue of this model is %.4f"%top_eigenvalues[-1])






In [ ]:
# finetuning


In [ ]:
# Comparison II (compute sharpness of finetuned solutions)